# Standarize series intensities

Trains landmark models for each series in the dataset

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, widgets, Layout, HBox, VBox, Label, interact, fixed
from IPython.display import display, clear_output
import itk
import SimpleITK as sitk
from concurrent.futures import ProcessPoolExecutor
import torchio as tio

In [ ]:
root_dir = '/local_ssd/practical_wise24/prostate_cancer/clean_dataset/' # should be this one
# root_dir = './clean_dataset/'

series = [
    'Pelvis_t2_spc_rst_tra_p2_iso',
    'Pelvis_t2_haste_fs_db_tra_p2_320',
    '*Pelvis_MRAC_PET_mlaa_siemens_Becken_1BP_15min_LM_AC Images',
    '*Pelvis_MRAC_PET_siemens_Becken_1BP_15min_LM_AC Images',
    'Pelvis_ep2d_diff_tra_ADC',
    'Pelvis_ep2d_diff_tra',
    'Pelvis_t1_tse_cor_p2',
]

In [ ]:
def get_series_file_paths(description):
    res = []
    for patient_folder in os.listdir(root_dir):
        files_path = os.path.join(root_dir, patient_folder)
        for mha_file_name in os.listdir(files_path):
            file_name_no_extension = mha_file_name.split('.')[0]
            if file_name_no_extension == description:
                res.append(os.path.join(files_path, mha_file_name))
    return res 

def get_patient_folder_from_path(path):
    sub_folders = path.split('/')
    for item in sub_folders:
        if 'BREST' in item:
            return item
    return None

In [ ]:
for description in series:
    serie_file_paths = get_series_file_paths(description)
    model_path = './landmarks/' + description + '.npy'
    # This step computes the landmarks used for standardization
    landmarks_npy = tio.HistogramStandardization.train(serie_file_paths, output_path=model_path)
    if not os.path.exists('./landmarks'):
        os.makedirs('./landmarks')
    np.save(model_path, landmarks_npy)